<a href="https://colab.research.google.com/github/ab17254/ce888/blob/main/Assignment/AIBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'aif360[all]'

     |████████████████████████████████| 184kB 6.7MB/s 
     |████████████████████████████████| 972kB 18.5MB/s 
     |████████████████████████████████| 276kB 38.4MB/s 
     |████████████████████████████████| 21.2MB 1.5MB/s 
     |████████████████████████████████| 2.6MB 38.2MB/s 
     |████████████████████████████████| 9.2MB 153kB/s 
     |████████████████████████████████| 358kB 38.4MB/s 
     |████████████████████████████████| 21.6MB 1.8MB/s 
     |████████████████████████████████| 3.4MB 47.3MB/s 
     |████████████████████████████████| 235kB 54.8MB/s 
     |████████████████████████████████| 747kB 47.1MB/s 
     |████████████████████████████████| 25.3MB 1.7MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=b8129ed33b67cd7a107b082d6195a2123e6fb9bdfcd5d33550bf0ddd2487d897
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py

In [3]:
!unzip /content/data.zip

Archive:  /content/data.zip
replace content/data/arrest_essex.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: content/data/arrest_essex.csv  
  inflating: content/data/arrest_suffolk.csv  
  inflating: content/data/arrest_manchester.csv  
  inflating: content/data/arrest_lancashire.csv  
  inflating: content/data/arrest_london.csv  
  inflating: content/data/data_all.csv  
  inflating: content/data/arrest_all.csv  
  inflating: content/data/arrest_wiltshire.csv  
  inflating: content/data/arrest_all_new.csv  


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import ReweighingMeta
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, PostProcessingMeta
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

In [2]:
df = pd.read_csv('content/data/arrest_all_new.csv')

In [3]:
df = df.sample(n=100000)

In [4]:
df['Time'] = df['Time'].str[:-3]

In [5]:
df_copy = df.copy()

In [6]:
df = df.set_index(['index','Ethnicity','Gender'])

In [7]:
df

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
30482,Unreported,Male,2017,Under 10 years,Warwickshire,-
34418,Black,Male,2011,All,Northumbria,338
204639,Mixed White and Black African,Male,2017,21 years and over,Metropolitan Police,504
49274,White,All,2014,10 - 17 years,Devon and Cornwall,1130
12314,Unreported,Male,2010,18 - 20 years,Hertfordshire,119
...,...,...,...,...,...,...
30476,Unreported,Male,2017,Under 10 years,South Yorkshire,-
92011,Pakistani,Female,2011,Under 10 years,Northamptonshire,-
25863,Asian,Female,2016,18 - 20 years,Leicestershire,4


In [8]:
X = df
y = df.iloc[:,0]

In [9]:
print(X.shape)
print(y.shape)

(100000, 4)
(100000,)


In [10]:
X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)

In [11]:
y = pd.Series(y.factorize(sort=True)[0], index=y.index)

In [13]:
# X = pd.get_dummies(X)

In [12]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.50)

In [13]:
X_train

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
2251,20,1,2007,18 - 20 years,South Yorkshire,761
98023,14,2,2010,18 - 20 years,All,668
42345,22,1,2011,10 - 17 years,Bedfordshire,1
40999,9,1,2011,Under 10 years,West Midlands,-
88130,0,0,2009,21 years and over,West Yorkshire,41960
...,...,...,...,...,...,...
33642,18,1,2008,18 - 20 years,Northumbria,-
17736,17,0,2006,Unknown,London City of,2
6651,17,1,2011,Under 10 years,Thames Valley,-


In [14]:
X_test

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
25611,13,0,2016,All,Gwent,103
75443,10,0,2012,21 years and over,West Mercia,167
84077,1,0,2010,All,Thames Valley,752
85489,1,0,2015,All,Gwent,59
35558,9,2,2009,10 - 17 years,Lincolnshire,7
...,...,...,...,...,...,...
1684,13,1,2007,21 years and over,Hertfordshire,67
51973,7,2,2015,18 - 20 years,Suffolk,N/A
15984,6,2,2012,All,Cambridgeshire,567


In [15]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(X_train)
X_train  = pd.DataFrame(ohe.transform(X_train), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), index=X_test.index)

In [16]:
y_train.head()

index  Ethnicity  Gender
2251   20         1         1
98023  14         2         4
42345  22         1         5
40999  9          1         5
88130  0          0         3
dtype: int64

In [17]:
X_train

,,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808,4809,4810,4811,4812,4813,4814,4815,4816,4817,4818,4819,4820,4821
index,Ethnicity,Gender,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2251,20,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98023,14,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42345,22,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40999,9,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88130,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33642,18,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17736,17,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6651,17,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
y_pred = LogisticRegression(solver='lbfgs').fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [21]:
disparate_impact_ratio(y_test, y_pred, prot_attr='Ethnicity')

1.0374052300125545

In [23]:
rew = ReweighingMeta(estimator=LogisticRegression(solver='lbfgs'))

params = {'estimator__C': [1, 10], 'reweigher__prot_attr': ['Ethnicity']}

clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.best_params_)

1.0
{'estimator__C': 1, 'reweigher__prot_attr': 'Ethnicity'}


In [25]:
disparate_impact_ratio(y_test, clf.predict(X_test), prot_attr='Ethnicity')


1.0374052300125545